In [1]:
import torch
import sys
from transformers import AutoProcessor, BarkModel
import logging
import queue
import asyncio
import numpy as np
import pyaudio
from IPython.display import Audio

logger = logging.getLogger(__name__)

if torch.cuda.is_available(): device = "cuda" 
elif sys.platform=='darwin' and torch.backends.mps.is_available(): device = "mps" 
else: device = "cpu"

processor = AutoProcessor.from_pretrained('suno/bark-small')
ttsmodel = BarkModel.from_pretrained('suno/bark-small')
ttsmodel.to(device)

/home/jrizzo/Projects/ai/ada/.venv/lib/python3.12/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


BarkModel(
  (semantic): BarkSemanticModel(
    (input_embeds_layer): Embedding(129600, 768)
    (position_embeds_layer): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-11): 12 x BarkBlock(
        (layernorm_1): BarkLayerNorm()
        (layernorm_2): BarkLayerNorm()
        (attn): BarkSelfAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (att_proj): Linear(in_features=768, out_features=2304, bias=False)
          (out_proj): Linear(in_features=768, out_features=768, bias=False)
        )
        (mlp): BarkMLP(
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (out_proj): Linear(in_features=3072, out_features=768, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
          (gelu): GELU(approximate='none')
        )
      )
    )
    (layernorm_final): BarkLayerNorm()
    (lm_head): Linear(in_features=76

In [2]:
SAMPLES_PER_SECOND=24000

class AsyncAudio:
    def __init__(self, 
                 chunk=1024,
                 format=pyaudio.paInt16,
                 channels=1,
                 rate=SAMPLES_PER_SECOND):
        self.chunk = chunk
        self.format = format
        self.channels = channels
        self.rate = rate
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(
            format=format,
            channels=channels,
            rate=rate,
            input=True,
            frames_per_buffer=chunk,
            stream_callback=self.callback,
        )
        self.queue = queue.Queue()
        self.is_recording = False
        self.is_receiving = False
        logger.info("AsyncMicrophone initialized")

    def callback(self, in_data, frame_count, time_info, status):
        if self.is_recording and not self.is_receiving:
            self.queue.put(in_data)
        return (None, pyaudio.paContinue)

    def start_recording(self):
        self.is_recording = True
        logger.info("Started recording")

    def stop_recording(self):
        self.is_recording = False
        logger.info("Stopped recording")

    def start_receiving(self):
        self.is_receiving = True
        self.is_recording = False
        logger.info("Started receiving assistant response")

    def stop_receiving(self):
        self.is_receiving = False
        logger.info("Stopped receiving assistant response")

    def get_audio_data(self):
        data = b""
        while not self.queue.empty():
            data += self.queue.get()
        return data if data else None

    def close(self):
        self.stream.stop_stream()
        self.stream.close()
        self.p.terminate()
        logger.info("AsyncMicrophone closed")

    async def play_audio(self, audio_data):
        p = pyaudio.PyAudio()
        stream = p.open(format=self.format,
                        channels=self.channels,
                        rate=self.rate,
                        output=True)
        stream.write(audio_data)

        # Add a small delay of silence at the end to prevent popping, and weird cuts off sounds
        silence_duration = 0.4
        silence_frames = int(self.rate * silence_duration)
        silence = b"\x00" * (
            silence_frames * self.channels * 2
        )  # 2 bytes per sample for 16-bit audio
        stream.write(silence)

        # Add a small pause before closing the stream to make sure the audio is fully played
        await asyncio.sleep(0.5)

        stream.stop_stream()
        stream.close()
        p.terminate()
        logger.debug("Audio playback completed")

    async def stop_running(self):
        self.exit_event.is_set()
    
    async def run(self):
        while not self.exit_event.is_set():
            await asyncio.sleep(0.1)
            if not self.is_receiving:
                self.play_audio(self.get_audio_data())
            else:
                await asyncio.sleep(0.1)


In [4]:
input("Press Enter to start recording...")
audio = AsyncAudio()
try:
    asyncio.run(audio.run())
    asyncio.sleep(10)
    audio.stop_running()
    audio.stop_recording()
    audio.close()
except KeyboardInterrupt:
    logger.info("Program terminated by user")
except Exception as e:
    logger.exception(f"An unexpected error occurred: {e}")

ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2793:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2793:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2793:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1377:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2793:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1377:(snd_func_refer) Unable to find definition 'cards.0.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:5204:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5727:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2